In [3]:
import os
import sys
import re
import datetime
import time
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import os
import subprocess

import numpy as np
import pandas as pd
import sys

In [4]:
#defining cutoffenergy and kpoing spacing arrays 
cutoff_energies = [600]

kpoint_spacings = [0.04]

pressures = [0, 4, 8]

elements = ["La", "N", "Bi"]

#confirm choices
print("Cutoff energies: ", cutoff_energies)
print("Kpoint Spacings: ", kpoint_spacings)
print("Pressures: ", pressures)
print("Elements: ", elements)

storage_directory = "/home/gridsan/tmackey/MyTools/CASTEP_Scripts/TestFolder/SuperCloudTest1/outputfiles" 

#cell_file_directory = "/home/gridsan/tmackey/Data/KISS_cell_files_First_Test_1_File" - 1 file
#cell_file_directory = "/home/gridsan/tmackey/Data/KISS_cell_files_First_Test_10etc" - 100 files
cell_file_directory = "/home/gridsan/tmackey/Data/KISS_cell_files_First_Test_10_files" #- 10 files
#cell_file_directory = "/home/gridsan/tmackey/Data/KISS_cell_files_First_Test" #- all files

#in the 10 file test, the compounds concerneded are :
#K3 In Se3 - not supposed to produce
#Nd5 (Mo3 O16) - not supposed to produce
#K3 (C O3) - not supposed to produce
#Cs In Cl3 - not supposed to produce
#Cs Cu Bi - not supposed to produce
#Ba (Si2 O5) - not supposed to produce
#Cu Br Se - not supposed to produce 
#Ce Cu5 Sn - not supposed to produce
#K6 Ge2 Te6 - not supposed to produce
#Os3 (C O)12 - not supposed to produce
#La In S - manually copied in, supposed to produce

CASTEP_tools_directory = "/home/gridsan/tmackey/MyTools/CASTEP_Scripts"

all_steps_script = "/home/gridsan/tmackey/MyTools/CASTEP_Scripts/all_steps.sh"

all_ternary_compounds_directory = "/home/gridsan/tmackey/Data/all_ternary_compounds.csv"

Cutoff energies:  [600]
Kpoint Spacings:  [0.04]
Pressures:  [0, 4, 8]
Elements:  ['La', 'N', 'Bi']


In [5]:
def remove_directory(directory_path):
    # Kill processes that are accessing .nfs files in the directory
    # try:
    #     result = subprocess.run(["lsof | grep .nfs | awk '{print $2}' | xargs kill -9"], shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    #     if result.returncode != 0:
    #         print(f"Failed to kill processes accessing .nfs files in {directory_path}: {result.stderr.decode('utf-8')}")
    # except Exception as e:
    #     print(f"Failed to kill processes accessing .nfs files in {directory_path}: {str(e)}")

    # Remove the directory
    try:
        result = subprocess.run(["rm", "-rf", directory_path], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        if result.returncode == 0:
            print(f"Successfully removed {directory_path}")
        else:
            print(f"Failed to remove {directory_path}: {result.stderr.decode('utf-8')}")
    except Exception as e:
        print(f"Failed to remove {directory_path}: {str(e)}")

remove_directory(storage_directory)

Successfully removed /home/gridsan/tmackey/MyTools/CASTEP_Scripts/TestFolder/SuperCloudTest1/outputfiles


In [6]:
def generate_command(all_steps_script, storage_directory, cutoff_energy, kpoint_spacing, pressure, cell_file_directory, all_ternary_compounds_directory, runtype, elements):

    command = [all_steps_script, "-d", storage_directory, "-e", str(cutoff_energy), "-k", str(kpoint_spacing), "-p", str(pressure), "-c", cell_file_directory, "-t", all_ternary_compounds_directory, "-r", runtype] + elements
    
    try:
        result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        if result.returncode == 0:
            print(f"Command executed successfully:\n{result.stdout.decode('utf-8')}")
        else:
            print(f"Failed to execute command: {result.stderr.decode('utf-8')}")
    except Exception as e:
        print(f"Failed to execute command: {str(e)}")

#test the initialization step 
generate_command(all_steps_script,storage_directory, cutoff_energies[0], kpoint_spacings[0], pressures[0], cell_file_directory, all_ternary_compounds_directory, "init", elements)
#what should happen:
#1. the new directory within the storage directory should be created

Command executed successfully:
starting reading_inputs
finished reading_inputs
start defining functions
/home/gridsan/tmackey/MyTools/CASTEP_Scripts/TestFolder/SuperCloudTest1/outputfiles/LaNBi-600-0.04-0/
Running initialization instructions



In [7]:
import decoration_and_initialization as deco

output_dir = storage_directory + "/" + elements[0] + elements[1] + elements[2] + "-" + str(cutoff_energies[0]) + "-" + str(kpoint_spacings[0]) + "-" + str(pressures[0]) 
#test
print(output_dir)
# "/home/gridsan/tmackey/MyTools/CASTEP_Scripts/TestFolder/SuperCloudTest1/outputfiles/LaPbBi-600-0.04-0"

/home/gridsan/tmackey/MyTools/CASTEP_Scripts/TestFolder/SuperCloudTest1/outputfiles/LaNBi-600-0.04-0


In [8]:
#update the deco package with importlib
import importlib
importlib.reload(deco)

deco.edit_all_cell_files(cell_file_directory, pressures[0], kpoint_spacings[0], elements, all_ternary_compounds_directory, output_dir)

#what should happen:
#1. 2 synthetic cell files should be created from the cell file in the input directory and copied to the output directory
#they should have the correct pressure, kpoint spacing and element substitutions made. their cutoff energies should be set to whatever the cutoff_energy is provided as 
#Pressure - checked
#Kpoints - checked

#for the folder of incomplete files (100etc) it correctly identified and labeled the cell files 

0.00138937130948245910.00243139979159430340.0041681139284473770.0031260854463355330.00034734282737061480.00347342827370614780.00069468565474122960.00208405696422368870.00277874261896491820.00382077110107676260.00.00173671413685307390.0010420284821118443












Error: code not found in all_ternaries.csv


In [9]:
#take in something like "Synth_20381_LaLaInBiSPb.cell" and return "20381"

def get_code_from_file_name(file_name):
    return file_name.split("_")[1]

codes = [get_code_from_file_name(file_name) for file_name in os.listdir(output_dir)]

#remove duplicates
codes = list(set(codes))

print(codes)

['20381']


In [10]:
import importlib
importlib.reload(deco)

for code in codes:
    deco.write_csv("checkpoint.csv", all_ternary_compounds_directory, code, os.path.join(cell_file_directory, code + ".cell"), output_dir, elements)

#this should produce an output in the test folder that looks like this for test folder 1 and the 10 file test folder 
# code,structure type,permutation,completion,error,error_type
# 20381,In5Nd4S13,"{'La': 'La', 'In': 'Pb', 'S': 'Bi'}",yes,,
# 20381,In5Nd4S13,"{'La': 'La', 'In': 'Bi', 'S': 'Pb'}",yes,,

In [11]:
generate_command(all_steps_script,storage_directory, cutoff_energies[0], kpoint_spacings[0], pressures[0], cell_file_directory, all_ternary_compounds_directory, "first_step_only", elements)

#tested with the 10 file folder 
#tested the different components by commenting out the unwanted stuff
#when focusing on the .param file it produced .param files that 
#(a) all lines unmodified except for the very end 
#(b) a line specifying the cutoff energy at the very end

#when tested on the .param file and the .sbatch file 
#(a) the .param file was modified as above
#(b) the sbatch file was modified correctly to make it runnable on supercloud 
#EXAMPLE: 
# #!/bin/bash
# #SBATCH -N 1
# #SBATCH -n 4
# #SBATCH --array=1
# #SBATCH --mem=4500
# #SBATCH --time=04:00:00
# #SBATCH --output="output/Synth_20381_LaLaInPbSBi_output.txt"
# #SBATCH --error="error/Synth_20381_LaLaInPbSBi_error.txt"

# echo "Date              = $(date)"
# echo "Hostname          = $(hostname -s)"
# echo "Working Directory = $(pwd)"

# module add mpi/openmpi-4.1.3

# export PATH=/home/gridsan/tmackey/support/CASTEP-21.11/bin/linux_x86_64_gfortran--mpi:$PATH

# export PATH=/home/gridsan/tmackey/support/CASTEP-21.11/bin/linux_x86_64_gfortran--serial:$PATH

# mpirun -np 4 castep.mpi Synth_20381_LaLaInPbSBi

#uncommented the sbatch submission line in all_steps and everthing looks like it's running smoothly!


###Transitioning to job array format 
#1. commented out the sbatch submission line in all_steps.sh
#it appears to have created something pretty decent: 


# #!/bin/bash
# #SBATCH -N 1
# #SBATCH -n 4
# #SBATCH --array=1-2
# #SBATCH --mem=4500
# #SBATCH --time=04:00:00
# #SBATCH --output="output/%A_output_%a.txt"
# #SBATCH --error="error/%A_error_%a.txt"

# echo "Date              = $(date)"
# echo "Hostname          = $(hostname -s)"
# echo "Working Directory = $(pwd)"

# module add mpi/openmpi-4.1.3

# export PATH=/home/gridsan/tmackey/support/CASTEP-21.11/bin/linux_x86_64_gfortran--mpi:$PATH
# export PATH=/home/gridsan/tmackey/support/CASTEP-21.11/bin/linux_x86_64_gfortran--serial:$PATH

# file_names=(Synth_*.cell)

# file=${file_names[$SLURM_ARRAY_TASK_ID-1]}

# echo "Running job ${SLURM_ARRAY_JOB_ID}.${SLURM_ARRAY_TASK_ID}"
# echo "File: $file"

# mpirun -np 4 castep.mpi ${file: -5}

#we'll see if it runs

#(2) uncommenting the sbatch submission line in all_steps.sh


Command executed successfully:
starting reading_inputs
finished reading_inputs
start defining functions
/home/gridsan/tmackey/MyTools/CASTEP_Scripts/TestFolder/SuperCloudTest1/outputfiles/LaNBi-600-0.04-0/
Running first step only instructions
Running generate_params.py
['output', 'Synth_20381_LaLaInBiSN.cell', 'Synth_20381_LaLaInNSBi.cell', 'error', 'checkpoint.csv']
starting_loop
Synth_20381_LaLaInBiSN.cell
task                 : geometryoptimization 

xc_functional        : LDA

spin_polarized       : false 

fix_occupancy        : false 

metals_method        : dm 

mixing_scheme        : pulay 

max_scf_cycles       : 300

opt_strategy         : speed 

page_wvfns           : 0 

num_dump_cycles      : 0 

backup_interval      : 39600

geom_method          : BFGS 

geom_max_iter        : 100

mix_history_length   : 100 

finite_basis_corr    : auto

fixed_npw            : false

write_cell_structure : true

write_bib            : false

write_otfg           : false

write_cst_esp  

In [22]:
import collect

import importlib
importlib.reload(collect)

collect.main(output_dir)

#tested on the 10-file folder
#this should update the checkpoint file and create a log file that updates the round column everytime its called

#after 1st push, checkpoint file looks good:
# code,structure type,permutation,completion,error,error_type
# 20381,In5Nd4S13,"{'La': 'La', 'In': 'Pb', 'S': 'Bi'}",incomplete,False,N/A
# 20381,In5Nd4S13,"{'La': 'La', 'In': 'Bi', 'S': 'Pb'}",incomplete,False,N/A

#log file also looks good:
# name,n-bands round,n-bands error,suggested extra bands
# Synth_20381_LaLaInPbSBi.cell,0,0,0
# Synth_20381_LaLaInBiSPb.cell,0,0,0

#after 2nd push, checkpoint file looks the same and log file is appropriately updated 
# Synth_20381_LaLaInPbSBi.cell,0,0,0
# Synth_20381_LaLaInBiSPb.cell,0,0,0
# Synth_20381_LaLaInPbSBi.cell,1,0,0
# Synth_20381_LaLaInBiSPb.cell,1,0,0

False
['Synth_20381_LaLaInBiSN.cell', 'Synth_20381_LaLaInNSBi.cell']
{'name': 'Synth_20381_LaLaInBiSN.cell', 'n-bands round': 0, 'n-bands error': 0, 'suggested extra bands': 0}
{'name': 'Synth_20381_LaLaInNSBi.cell', 'n-bands round': 0, 'n-bands error': 0, 'suggested extra bands': 0}
[{'name': 'Synth_20381_LaLaInBiSN.cell', 'n-bands round': 0, 'n-bands error': 0, 'suggested extra bands': 0}, {'name': 'Synth_20381_LaLaInNSBi.cell', 'n-bands round': 0, 'n-bands error': 0, 'suggested extra bands': 0}]
['Synth_20381_LaLaInBiSN.cell', 'Synth_20381_LaLaInNSBi.cell']
[{'name': 'Synth_20381_LaLaInBiSN.cell', 'completion': 'incomplete', 'error': False, 'error type': 'N/A'}, {'name': 'Synth_20381_LaLaInNSBi.cell', 'completion': 'incomplete', 'error': False, 'error type': 'N/A'}]
[['code', 'structure type', 'permutation', 'completion', 'error', 'error_type'], ['20381', 'In5Nd4S13', "{'La': 'La', 'In': 'N', 'S': 'Bi'}", 'yes', '', ''], ['20381', 'In5Nd4S13', "{'La': 'La', 'In': 'Bi', 'S': 'N'}", '